# Challenge

In [69]:
%run utils.ipynb

In [70]:
import pandas as pd
import numpy as np
import os
import sys
# from utils import *
import matplotlib.pyplot as plt
%matplotlib inline
import dill
from scipy import stats
from collections import Counter

In [71]:
import sklearn
from sklearn.svm import OneClassSVM
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, top_k_accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, FunctionTransformer, OrdinalEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline, make_pipeline


In [72]:
_, seg_df = load_user_data()
print(seg_df.shape)
y_seg = pd.read_csv('challengeToFill.csv').drop(columns=['Unnamed: 0']).T.reset_index(drop=True).T
y_seg_train = y_seg.iloc[:, :TRAIN_SEG]
y_seg_test = y_seg.iloc[:, TRAIN_SEG:]
train_df = seg_df.iloc[:, :TRAIN_SEG]
test_df = seg_df.iloc[:, TRAIN_SEG:]
submission_df = seg_df.iloc[LABELED_USERS:, TRAIN_SEG:]
print(train_df.shape)
print(test_df.shape)

(40, 150)
(40, 50)
(40, 100)


In [76]:
def tag_user_segments(user_id):
    other_users = 50
    y_train = np.concatenate([np.zeros(50), np.ones(39*other_users)]).astype(int)
    y_test = y_seg_test.iloc[user_id, :].to_numpy().flatten()
    X_train = train_df.iloc[[user_id]]
    X_test = pd.DataFrame(test_df.iloc[[user_id]].to_numpy().flatten())
    for i in range(40):
        if i!=user_id:
            X_train = pd.DataFrame(np.concatenate((X_train.to_numpy().flatten(), train_df.iloc[i, :other_users].values)))
    return X_train, X_test, y_train, y_test

In [80]:
X_train_user

,0
0,"[cat, nawk, nawk, uname, pwd, echo, echo, ksh,..."
1,"[nawk, nawk, uname, pwd, echo, echo, ksh, unam..."
2,"[nawk, uname, pwd, echo, echo, ksh, uname, stt..."
3,"[uname, pwd, echo, echo, ksh, uname, stty, tse..."
4,"[pwd, echo, echo, ksh, uname, stty, tset, ksh,..."
...,...
1995,"[hostname, id, nawk, getopt, true, true, grep,..."
1996,"[id, nawk, getopt, true, true, grep, date, lp,..."
1997,"[nawk, getopt, true, true, grep, date, lp, fin..."
1998,"[getopt, true, true, grep, date, lp, find, exp..."


## TF-IDF Per User

In [48]:
pd.DataFrame.sparse.from_spmatrix(tfidf_vec.fit_transform([' '.join(x) for x in train_df.iloc[user_id].to_numpy().flatten()]), columns=count_vec.get_feature_names_out()).sparse.to_dense()

,Sqpe sendmail movemail,cat nawk nawk,echo echo ksh,echo ksh gzip,echo ksh ln,echo ksh ls,echo ksh mkdir,echo ksh uname,find ls ls,ghostvie uname uname,...,uname pwd echo,uname stty tset,uname uname pwd,whoami ksh sendmail,xgvis Sqpe sendmail,xgvis ln sh,xgvis ls ln,xgvis ls ls,xgvis ls mkdir,xgvis sh sh
0,0.000000,0.226925,0.219796,0.144453,0.074611,0.074611,0.074611,0.156579,0.087341,0.000000,...,0.219796,0.189217,0.194692,0.000000,0.000000,0.074611,0.074611,0.000000,0.074611,0.126327
1,0.000000,0.135827,0.222750,0.146395,0.075613,0.075613,0.075613,0.158683,0.088515,0.000000,...,0.222750,0.191760,0.197309,0.000000,0.000000,0.075613,0.075613,0.000000,0.075613,0.128025
2,0.000000,0.137357,0.225260,0.148044,0.076465,0.076465,0.076465,0.160471,0.089512,0.000000,...,0.225260,0.193920,0.199531,0.000000,0.000000,0.076465,0.076465,0.000000,0.076465,0.129467
3,0.000000,0.139139,0.228182,0.149964,0.077457,0.077457,0.077457,0.162553,0.090673,0.000000,...,0.228182,0.196436,0.202120,0.000000,0.000000,0.077457,0.077457,0.082153,0.077457,0.131146
4,0.000000,0.139020,0.227987,0.149836,0.077391,0.077391,0.077391,0.162413,0.090596,0.000000,...,0.216057,0.196268,0.201947,0.000000,0.000000,0.077391,0.077391,0.082083,0.077391,0.131034
5,0.000000,0.138881,0.227760,0.149687,0.077314,0.077314,0.077314,0.162252,0.090506,0.000000,...,0.215842,0.196073,0.201746,0.000000,0.000000,0.077314,0.077314,0.082001,0.077314,0.130904
6,0.000000,0.138786,0.215693,0.149584,0.077261,0.077261,0.077261,0.162140,0.090443,0.000000,...,0.215693,0.195938,0.201607,0.000000,0.000000,0.077261,0.077261,0.081945,0.077261,0.130814
7,0.000000,0.138786,0.215693,0.149584,0.077261,0.077261,0.077261,0.130814,0.090443,0.000000,...,0.215693,0.195938,0.201607,0.000000,0.000000,0.077261,0.077261,0.081945,0.077261,0.130814
8,0.000000,0.139976,0.217543,0.150866,0.077923,0.077923,0.077923,0.131935,0.091219,0.000000,...,0.217543,0.197618,0.203336,0.000000,0.000000,0.077923,0.077923,0.082647,0.077923,0.163530
9,0.000000,0.141357,0.219690,0.152355,0.078692,0.078692,0.078692,0.133238,0.092119,0.000000,...,0.219690,0.117868,0.205342,0.000000,0.000000,0.078692,0.078692,0.083463,0.078692,0.165144


In [52]:
count_vec = CountVectorizer(lowercase=False, vocabulary=list(tfidf_vec.vocabulary_.keys()))

In [116]:
X = pd.DataFrame()

for user_id in range(40):
    
    # print(f'User {user_id}')

    # tfidf_vec = TfidfVectorizer(lowercase=False , ngram_range=(3,3), max_features=1000, sublinear_tf=True)

    func = FunctionTransformer(lambda y: [' '.join(x) for x in y.iloc[user_id].to_numpy().flatten()])

    # tfidf_vec.fit_transform(func.fit_transform(train_df))
    
    # print(tfidf_vec.vocabulary_)

    count_vec = CountVectorizer(lowercase=False, max_features=1000, ngram_range=(3,3))

    clf = RandomForestClassifier(random_state=42, n_estimators=80)

    pipe = make_pipeline(func, count_vec, clf)
    
    X_train_user, X_test_user, y_train_user, y_test_user = tag_user_segments(user_id)

    pipe.fit(train_df)
    
    pipe.predict(test_df)
    
    # X_test = pd.DataFrame.sparse.from_spmatrix(pipe.predict(test_df), columns=count_vec.get_feature_names_out()).sparse.to_dense()

IndexError: positional indexers are out-of-bounds

In [115]:
X = pd.DataFrame()

for user_id in range(10):

    tom = pd.DataFrame.sparse.from_spmatrix(pipe.transform(test_df), columns=count_vec.get_feature_names_out()).sparse.to_dense()

    X_test = pd.concat([X_test, tom]).fillna(0)

X_test = X_test.reset_index(drop=True)
X_test

,FIFO cat date,FIFO rm UNLOCK,LOCK sh mimencod,LOCK true ls,MediaMai launchef launchef,MediaMai netscape netscape,UNLOCK rmdir generic,awk cat post,cat date generic,cat file post,...,tcppost sh LOCK,true grep date,true ls sed,true true grep,userenv wait4wm xhost,wait4wm xhost xsetroot,xhost xsetroot reaper,xrdb cpp sh,xrdb mkpts env,xsetroot reaper cat
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,1,1,1,...,1,0,1,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,1,1,1,...,1,0,1,0,0,0,0,0,0,0
3,1,1,1,1,1,1,1,1,1,1,...,1,0,1,0,0,0,0,0,0,0
4,1,1,1,1,1,1,1,1,1,1,...,1,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,1,1,1,1,1
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,1,1,1,1
97,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,1,1,1,1
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,1,1,1,1


In [110]:
def tag_user_segments(user_id):
    other_users = 50
    y_train = np.concatenate([np.zeros(50), np.ones(39*other_users)]).astype(int)
    y_test = y_seg_test.iloc[user_id, :].to_numpy().flatten()
    X_train = X.iloc[[user_id]]
    X_test = pd.DataFrame(test_df.iloc[[user_id]].to_numpy().flatten())
    for i in range(40):
        if i!=user_id:
            X_train = pd.DataFrame(np.concatenate((X_train.to_numpy().flatten(), train_df.iloc[i, :other_users].values)))
    return X_train, X_test, y_train, y_test

In [111]:
total_score = 0

for user_id in range(LABELED_USERS):
        
    X_train_user, X_test_user, y_train_user, y_test_user = tag_user_segments(user_id)

    clf = RandomForestClassifier(n_estimators=80, random_state=42, class_weight='balanced')
    
    pipe = clf
    
    clf.fit(X_train_user, y_train_user)

    test_preds = clf.predict(X_test_user)
    
    test_probas = clf.predict_proba(X_test_user)

    total_score += get_metrics(y_test_user, test_preds)
    
print(f'\n{total_score}')

ValueError: setting an array element with a sequence.